In [1]:
import anndata
import numpy as np
import scanpy as sc
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [2]:
dataset = 'Erythroid_Human'
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')
adata.obs['clusters'] = adata.obs['type2'].to_numpy()

In [ ]:
vv.preprocess(adata, n_gene=2000, min_shared_counts=20)
adata.write_h5ad(f'data/{dataset}_pp.h5ad')

In [3]:
model_path_base = f'checkpoints/{dataset}'
figure_path_base = f'figures/{dataset}'
data_path = f'data/velovae/continuous/{dataset}'
gene_plot = ['CNN3','CYR61','ABCG2','HBA1']

# Vanilla VAE

In [5]:
figure_path = f'{figure_path_base}/Vanilla'
model_path = f'{model_path_base}/Vanilla'

torch.manual_seed(2022)
np.random.seed(2022)
vanilla_vae = vv.VanillaVAE(adata, 
                            tmax=20, 
                            device='cuda:0',
                            init_method='steady')
vanilla_vae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)
vanilla_vae.save_model(model_path, 'encoder', 'decoder')
vanilla_vae.save_anndata(adata, 'vanilla', data_path, file_name=f'{dataset}.h5ad')

Initialization using the steady-state and dynamical models.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
------------------------- Train a Vanilla VAE -------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
Total Number of Iterations Per Epoch: 197, test iteration: 392
********* Early Stop Triggered at epoch 160. *********
*********              Finished. Total Time =   0 h : 12 m : 36 s             *********
Final: Train ELBO = 454.334,           Test ELBO = 459.611


# VeloVAE

In [6]:
figure_path = f'{figure_path_base}/VeloVAE'
model_path = f'{model_path_base}/VeloVAE'

torch.manual_seed(2022)
np.random.seed(2022)
vae = vv.VAE(adata, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0')

vae.train(adata, gene_plot=gene_plot, plot=False, figure_path=figure_path)
vae.save_model(model_path, 'encoder', 'decoder')
vae.save_anndata(adata, 'velovae', data_path, file_name=f'{dataset}.h5ad')

Initialization using the steady-state and dynamical models.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 197, test iteration: 392
*********       Stage 1: Early Stop Triggered at epoch 394.       *********
*********                      Stage  2                       *********
Cell-wise KNN Estimation.


  0%|          | 0/35877 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.070
Average Set Size: 1302
Finished. Actual Time:   0 h :  0 m : 50 s
*********       Stage 2: Early Stop Triggered at epoch 490.       *********
*********              Finished. Total Time =   0 h : 28 m : 18 s             *********
Final: Train ELBO = 892.100,           Test ELBO = 892.500


# Full VB

In [7]:
figure_path = f'{figure_path_base}/FullVB'
model_path = f'{model_path_base}/FullVB'

torch.manual_seed(2022)
np.random.seed(2022)
full_vb = vv.VAEFullVB(adata, 
                      tmax=20, 
                      dim_z=5, 
                      device='cuda:0')

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'fullvb', data_path, file_name=f'{dataset}.h5ad')

Initialization using the steady-state and dynamical models.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 197, test iteration: 392
*********       Stage 1: Early Stop Triggered at epoch 239.       *********
*********                      Stage  2                       *********
Cell-wise KNN Estimation.


  0%|          | 0/35877 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.059
Average Set Size: 1068
Finished. Actual Time:   0 h :  0 m : 47 s
*********       Stage 2: Early Stop Triggered at epoch 313.       *********
*********              Finished. Total Time =   0 h : 23 m :  0 s             *********
Final: Train ELBO = 860.076,           Test ELBO = 857.640


# Branching ODE

In [8]:
figure_path = f'{figure_path_base}/BrODE'
model_path = f'{model_path_base}/BrODE'

torch.manual_seed(2022)
np.random.seed(2022)
brode = vv.BrODE(adata, 'clusters', 'fullvb_time', 'fullvb_z')

brode.print_weight()

brode.train(adata, 'fullvb_time', 'clusters', plot=False, gene_plot=gene_plot, figure_path=figure_path)

brode.save_model(model_path, 'brode')
brode.save_anndata(adata, 'brode', data_path, file_name=f'{dataset}.h5ad')

Graph Partition
Number of partitions:  2
Computing type-to-type transition probability
Obtaining the MST in each partition
Initialization using type-specific dynamical model.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

                 Early Erythroid  Late Erythroid  MEMP  Mid  Erythroid
Early Erythroid              0.0             0.0   0.0             1.0
Late Erythroid               0.0             1.0   0.0             0.0
MEMP                         0.0             0.0   0.0             1.0
Mid  Erythroid               0.0             1.0   0.0             0.0
------------------------ Train a Branching ODE ------------------------
                 Early Erythroid  Late Erythroid  MEMP  Mid  Erythroid
Early Erythroid              0.0             0.0   0.0             1.0
Late Erythroid               0.0             1.0   0.0             0.0
MEMP                         0.0             0.0   0.0             1.0
Mid  Erythroid               0.0             1.0   0.0             0.0
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
**

# Evaluation

In [ ]:
cluster_edges = [('MEMP', 'Early Erythroid'), 
                 ('Early Erythroid', 'Mid  Erythroid'), 
                 ('Mid  Erythroid', 'Late Erythroid')]
vv.post_analysis(adata,
                 'eval',
                 ['Vanilla VAE','VeloVAE','FullVB','BrODE'],
                 ['vanilla','velovae','fullvb','brode'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)